# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 13 2023 8:30AM,259123,10,37832,"Digital Brands, LLC",Executing
1,Mar 13 2023 8:30AM,259123,10,37833,"Digital Brands, LLC",Released
2,Mar 10 2023 3:56PM,259129,15,PNC432642A,"Person & Covey, Inc.",Released
3,Mar 10 2023 3:46PM,259128,15,30014586,"Alliance Pharma, Inc.",Released
4,Mar 10 2023 3:46PM,259128,15,9183162-T,"Brookfield Pharmaceuticals, LLC",Released
5,Mar 10 2023 2:58PM,259121,10,MSP219423,Methapharm-G,Released
6,Mar 10 2023 2:53PM,259120,21,0848800,"NBTY Global, Inc.",Executing
7,Mar 10 2023 2:53PM,259119,10,MSP218388-BO,Methapharm-G,Released
8,Mar 10 2023 2:49PM,259118,10,Snap-9252961,"Snap Medical Industries, LLC",Released
9,Mar 10 2023 2:46PM,259117,10,MSP218278-BO,Methapharm-G,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,259121,Released,1
36,259123,Executing,1
37,259123,Released,1
38,259128,Released,2
39,259129,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259120,NaN,1.0,NaN
259121,NaN,NaN,1.0
259123,NaN,1.0,1.0
259128,NaN,NaN,2.0
259129,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259023,0.0,1.0,3.0
259024,0.0,0.0,1.0
259033,0.0,3.0,2.0
259057,19.0,11.0,3.0
259071,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259023,0,1,3
259024,0,0,1
259033,0,3,2
259057,19,11,3
259071,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259023,0,1,3
1,259024,0,0,1
2,259033,0,3,2
3,259057,19,11,3
4,259071,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259023,,1,3
1,259024,,,1
2,259033,,3,2
3,259057,19,11,3
4,259071,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 13 2023 8:30AM,259123,10,"Digital Brands, LLC"
2,Mar 10 2023 3:56PM,259129,15,"Person & Covey, Inc."
3,Mar 10 2023 3:46PM,259128,15,"Alliance Pharma, Inc."
4,Mar 10 2023 3:46PM,259128,15,"Brookfield Pharmaceuticals, LLC"
5,Mar 10 2023 2:58PM,259121,10,Methapharm-G
6,Mar 10 2023 2:53PM,259120,21,"NBTY Global, Inc."
7,Mar 10 2023 2:53PM,259119,10,Methapharm-G
8,Mar 10 2023 2:49PM,259118,10,"Snap Medical Industries, LLC"
9,Mar 10 2023 2:46PM,259117,10,Methapharm-G
10,Mar 10 2023 2:45PM,259116,10,Methapharm-G


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 13 2023 8:30AM,259123,10,"Digital Brands, LLC",,1,1
1,Mar 10 2023 3:56PM,259129,15,"Person & Covey, Inc.",,,1
2,Mar 10 2023 3:46PM,259128,15,"Alliance Pharma, Inc.",,,2
3,Mar 10 2023 3:46PM,259128,15,"Brookfield Pharmaceuticals, LLC",,,2
4,Mar 10 2023 2:58PM,259121,10,Methapharm-G,,,1
5,Mar 10 2023 2:53PM,259120,21,"NBTY Global, Inc.",,1,
6,Mar 10 2023 2:53PM,259119,10,Methapharm-G,,,1
7,Mar 10 2023 2:49PM,259118,10,"Snap Medical Industries, LLC",,,1
8,Mar 10 2023 2:46PM,259117,10,Methapharm-G,,,1
9,Mar 10 2023 2:45PM,259116,10,Methapharm-G,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 8:30AM,259123,10,"Digital Brands, LLC",1,1,
1,Mar 10 2023 3:56PM,259129,15,"Person & Covey, Inc.",1,,
2,Mar 10 2023 3:46PM,259128,15,"Alliance Pharma, Inc.",2,,
3,Mar 10 2023 3:46PM,259128,15,"Brookfield Pharmaceuticals, LLC",2,,
4,Mar 10 2023 2:58PM,259121,10,Methapharm-G,1,,
5,Mar 10 2023 2:53PM,259120,21,"NBTY Global, Inc.",,1,
6,Mar 10 2023 2:53PM,259119,10,Methapharm-G,1,,
7,Mar 10 2023 2:49PM,259118,10,"Snap Medical Industries, LLC",1,,
8,Mar 10 2023 2:46PM,259117,10,Methapharm-G,1,,
9,Mar 10 2023 2:45PM,259116,10,Methapharm-G,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 8:30AM,259123,10,"Digital Brands, LLC",1,1,
1,Mar 10 2023 3:56PM,259129,15,"Person & Covey, Inc.",1,,
2,Mar 10 2023 3:46PM,259128,15,"Alliance Pharma, Inc.",2,,
3,Mar 10 2023 3:46PM,259128,15,"Brookfield Pharmaceuticals, LLC",2,,
4,Mar 10 2023 2:58PM,259121,10,Methapharm-G,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 8:30AM,259123,10,"Digital Brands, LLC",1.0,1.0,NaN
1,Mar 10 2023 3:56PM,259129,15,"Person & Covey, Inc.",1.0,NaN,NaN
2,Mar 10 2023 3:46PM,259128,15,"Alliance Pharma, Inc.",2.0,NaN,NaN
3,Mar 10 2023 3:46PM,259128,15,"Brookfield Pharmaceuticals, LLC",2.0,NaN,NaN
4,Mar 10 2023 2:58PM,259121,10,Methapharm-G,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 8:30AM,259123,10,"Digital Brands, LLC",1.0,1.0,0.0
1,Mar 10 2023 3:56PM,259129,15,"Person & Covey, Inc.",1.0,0.0,0.0
2,Mar 10 2023 3:46PM,259128,15,"Alliance Pharma, Inc.",2.0,0.0,0.0
3,Mar 10 2023 3:46PM,259128,15,"Brookfield Pharmaceuticals, LLC",2.0,0.0,0.0
4,Mar 10 2023 2:58PM,259121,10,Methapharm-G,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4145775,32.0,1.0,0.0
12,259091,0.0,1.0,0.0
15,2331696,20.0,15.0,0.0
18,259084,1.0,0.0,0.0
19,1036348,8.0,14.0,19.0
21,518191,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4145775,32.0,1.0,0.0
1,12,259091,0.0,1.0,0.0
2,15,2331696,20.0,15.0,0.0
3,18,259084,1.0,0.0,0.0
4,19,1036348,8.0,14.0,19.0
5,21,518191,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,32.0,1.0,0.0
1,12,0.0,1.0,0.0
2,15,20.0,15.0,0.0
3,18,1.0,0.0,0.0
4,19,8.0,14.0,19.0
5,21,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,32.0
1,12,Released,0.0
2,15,Released,20.0
3,18,Released,1.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,21
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,19.0,0.0
Executing,1.0,1.0,15.0,0.0,14.0,2.0
Released,32.0,0.0,20.0,1.0,8.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,21
0,Completed,0.0,0.0,0.0,0.0,19.0,0.0
1,Executing,1.0,1.0,15.0,0.0,14.0,2.0
2,Released,32.0,0.0,20.0,1.0,8.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,21
0,Completed,0.0,0.0,0.0,0.0,19.0,0.0
1,Executing,1.0,1.0,15.0,0.0,14.0,2.0
2,Released,32.0,0.0,20.0,1.0,8.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()